---

## Deep Learning Coding Project 2: Image Classification

Before we start, please put your **Chinese** name and student ID in following format:

黄浩峰, 2022011339

YOUR ANSWER HERE

# Introduction

We will use Python 3, [NumPy](https://numpy.org/), and [PyTorch](https://pytorch.org/) for this coding project. The example code has been tested under the latest stable release version.

### Task

In this notebook, you need to train a model to classify images. Given an image, you need to distinguish its category,
e.g., whether it is a horse or an automobile. There are a total of 10 classes:
airplane, automobile, bird, cat, deer, dog, frog, horse, ship and truck. We
release around 90,000 images for training and around 90,000 images for validation. Each image has
a shape of (3, 32, 32). We will evaluate your model in around 90,000 images on the test set.

Download the dataset from [here](https://cloud.tsinghua.edu.cn/d/7a87067e20b445afb384/) and organize them into a folder named "cinic_10".

### Coding

We provide a code template. You can add new cells and modify our example to train your own model. To run this code, you should:

+ implement your model (named `Net`) in `model.py`;
+ implement your dataset in this notebook;
+ implement your training loop in this notebook;
+ include necessary tricks in your training pipeline, e.g., changing batch size, data augmentation, learning rate schedule, etc. (**Feel free to modify this notebook!**)

Your final submitted model should not be larger than **50M**. **Using any pretrained model is NOT permitted**. The results of your model should be reproducible.
Besides, before you submit your result, **make sure you can test your model using our evaluation cell.** Name your best model "cinic10_best.pth".

### Report & Submission

Your report should include:

1. the details of your model
2. all the hyper-parameters
3. all the tricks or training techniques you use
4. the training curve of your submitted model.

Reporting additional ablation studies and how you improve your model are also encouraged.

You should submit:

+ all codes
+ the model checkpoint (only "cinic10_best.pth")
+ your report (a separate "pdf")

to web learning. We will use the evaluation code in this notebook to evaluate your model on the test set.

### Grading

We will grade this coding project based on the performance of your model (70%) and your report (30%). Regarding the evaluation metric of your model, assume your test accuracy is $X$, then your score is

$\frac{min(X,H)−0.6}{H−0.6}×7$

where $H$ is accuracy of the model trained by TAs and $H=0.85$, i.e., you will get the full score if your test accuracy is above 85%.

**Bonus**: The best submission with the highest testing accuracy will get 1 bonus point for the final course grade.

**Avoid plagiarism! Any student who violates academic integrity will be seriously dealt with and receive an F for the course.**

## Code Template

We have masked the the training loop in this notebook for you to complete. You should also overwrite "model.py" and implement your own model.

In [1]:
%load_ext autoreload
%autoreload 2

### Setup Code

If you use Colab in this coding project, please uncomment the code, fill the `GOOGLE_DRIVE_PATH_AFTER_MYDRIVE` and run the following cells to mount your Google drive. Then, the notebook can find the required file. If you run the notebook locally, you can skip the following cells.

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# import os

# # TODO: Fill in the Google Drive path where you uploaded the assignment
# # Example: If you create a 2023SP folder and put all the files under CP2 folder, then '2023SP/CP2'
# # GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '2023SP/CP2'
# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = None 
# GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
# print(os.listdir(GOOGLE_DRIVE_PATH))

In [4]:
# import sys
# sys.path.append(GOOGLE_DRIVE_PATH)

In [5]:
from evaluation import evaluation
from dataset import CINIC10 # this should be implemented by yourself
from model import Net # this should be implemented by yourself

### Enjoy Your Coding Time!

In [6]:
import math
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms



def set_seed(seed):
    seed = int(seed)
    if seed < 0 or seed > (2**32 - 1):
        raise ValueError("Seed must be between 0 and 2**32 - 1")
    else:
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True


device = 'cuda' if torch.cuda.is_available() else 'cpu'
set_seed(16)

### Train Setting
Below defines the training setting used in train loop, the given `train_transform` may lead to poor performance, feel free to try different transforms combinations to achieve better results.

In [8]:
data_root_dir = './cinic10/'

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
# train_transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Resize((32, 32)),
#     transforms.Normalize(mean=mean,std=std),
#     transforms.RandomVerticalFlip(p=0.5),
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomRotation([-90, 90]),
#     transforms.RandomGrayscale(p=0.5),
#     transforms.RandomAutocontrast(),
#     transforms.RandomInvert(),
# ])
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((32, 32))]
)
# valid_transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Resize((32, 32)),
#     transforms.Normalize(mean=mean,std=std),
# ])
valid_transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Resize((32, 32))])

trainset = CINIC10(root=data_root_dir,
                      split="train", transform=train_transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=512, shuffle=True, num_workers=32, pin_memory=True)

validset = CINIC10(root=data_root_dir,
                      split='valid', transform=valid_transform)
validloader = torch.utils.data.DataLoader(
    validset, batch_size=1024, shuffle=False, num_workers=8)

net = Net()
net(trainset[0][0])
print("number of trained parameters: %d" % (
    sum([param.nelement() for param in net.parameters() if param.requires_grad])))
print("number of total parameters: %d" %
      (sum([param.nelement() for param in net.parameters()])))

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 100, verbose=False, eta_min=1e-5)

net.to(device)

number of trained parameters: 18326
number of total parameters: 18326


Net(
  (network): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=36, out_features=120, bias=True)
    (8): ReLU()
    (9): Linear(in_features=120, out_features=84, bias=True)
    (10): ReLU()
    (11): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [ ]:
model_dir = '.'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
torch.save(net, os.path.join(model_dir, 'cinic10_0.pth'))

# check the model size
os.system(' '.join(['du', '-h', os.path.join(model_dir, 'cinic10_0.pth')]))

In [ ]:
def train():
    print(f'train on {device}')
    ##############################################################################
    #                  TODO: You need to complete the code here                  #
    ##############################################################################
    # YOUR CODE HERE
    raise NotImplementedError()
    ##############################################################################
    #                              END OF YOUR CODE                              #
    ##############################################################################
train()
# evaluation(net, validloader, device)

## Evaluation

Before submission, please run the following cell to make sure your model can be correctly graded.

In [ ]:
!python3 evaluation.py